# Introduction
The purpose of this notebook is to create an updated version of sample metadata file with:
- ox_codes instead of Individual ID
- remove year from unselected dup (for panoptes)
- add GPS coordinates
- create new files for Vikki
- create new Panoptes file for Ben

In [1]:
%run _standard_imports.ipynb

python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.2
scipy 0.18.1
pandas 0.19.0
numexpr 2.6.1
pysam 0.8.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.2.0
h5py 2.6.0
tables 3.3.0


In [46]:
panoptes_previous_metadata_fn = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161214_samples_which_release/pf_6_0_panoptes_samples_final_20170124.txt.gz"
panoptes_final_metadata_fn = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161214_samples_which_release/pf_6_0_panoptes_samples_final_20170206.txt.gz"
sites_fn = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161214_samples_which_release/pf_6_0_sites_20170206.xlsx"
study_summary_fn = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161214_samples_which_release/study_summary_20170206.xlsx"
samples_by_study_fn = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161214_samples_which_release/samples_by_study_20170206.xlsx"
sample_6_0_fn = "/nfs/team112_internal/production/release_build/Pf/6_0_release_packages/Pf_60_sample_metadata.txt"
panoptes_samples_fn = "/nfs/team112_internal/production/release_build/Pf/6_0_release_packages/Pf_60_samples_panoptes_20170206.txt"


In [15]:
tbl_panoptes_samples_previous = (
    etl
    .fromtsv(panoptes_previous_metadata_fn)
    .convertnumbers()
)
print(len(tbl_panoptes_samples_previous.data()))
print(len(tbl_panoptes_samples_previous.distinct('Sample').data()))
tbl_panoptes_samples_previous

7182
7182


Sample,OxfordSrcCode,OxfordDonorCode,Individual_ID,AlfrescoStudyCode,Type,InV5.1,InV5,SubContinentCode,SubContinentName,RegionCode,CountryCode,CountryName,Site,Year,Date,HasDuplicate,DiscardAsDuplicate,run_accessions,pc_pass_missing,pc_genome_covered_at_1x,Year_source,sort_AlfrescoStudyCode,IndividualGroup
PM0001-C,215C,215C,215C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR012350,ERR012360,ERR012305",0.15139786050931692,0.9696,study,1001-PF-ML-DJIMDE,1
PM0002-C,217C,217C,217C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR012520,ERR012331,ERR012327",0.28678549872221293,0.864,study,1001-PF-ML-DJIMDE,2
PM0003-C,520C,520C,520C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR012492,ERR012457,ERR012532",0.1662372060768727,0.9586,study,1001-PF-ML-DJIMDE,3
PM0004-C,A336TPI7C,A336TPI7C,A336TPI7C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR015438,ERR015329,ERR015422,ERR015315",0.11811536041300132,0.9648,study,1001-PF-ML-DJIMDE,4
PM0103-C,ALF003,ALF003,ALF003,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Faladje,2007,2007,False,False,ERR199991,0.13763789184399441,0.9485,study,1001-PF-ML-DJIMDE,5


In [9]:
tbl_duplicates_ox_code = (
    tbl_panoptes_samples_previous
    .aggregate('IndividualGroup', etl.strjoin(','), 'Sample')
    .rename('value', 'AllSamplesThisIndividual')
)

In [11]:
tbl_duplicates_ox_code.select(lambda rec: len(rec['AllSamplesThisIndividual']) > 10)

IndividualGroup,AllSamplesThisIndividual
268,"PE0017-C,PE0017-CW"
273,"PE0009-C,PE0009-CW"
431,"QE0359-C,QE0385-C"
468,"QC0116-C,QC0117-C"
650,"PH0027-C,PH0027-Cx"


In [14]:
tbl_sites = (
    etl.fromxlsx(sites_fn)
    .rename('Source', 'Site_source')
)
print(len(tbl_sites.data()))
print(len(tbl_sites.distinct('Site').data()))
tbl_sites

79
79


Site,CountryCode,Lat,Long,Site_source
Bandarban,BD,21.756949,92.383575,Rob
Ramu,BD,21.43199,92.101,Rob
Bobo-Dioulasso,BF,11.177055,-4.300861,Rob
Homel,BJ,6.3607027,2.4381709,http://placesmap.net/BJ/HOMEL-89462/
Kinshasa,CD,-4.36939,15.320977,Rob


In [17]:
tbl_panoptes_samples_final = (
    tbl_panoptes_samples_previous
    .join(tbl_duplicates_ox_code, key='IndividualGroup')
    .join(tbl_sites, key='Site')
    .sort(['AlfrescoStudyCode', 'Sample'])
)
print(len(tbl_panoptes_samples_final.data()))
print(len(tbl_panoptes_samples_final.distinct('Sample').data()))
tbl_panoptes_samples_final

7182
7182


Sample,OxfordSrcCode,OxfordDonorCode,Individual_ID,AlfrescoStudyCode,Type,InV5.1,InV5,SubContinentCode,SubContinentName,RegionCode,CountryCode,CountryName,Site,Year,Date,HasDuplicate,DiscardAsDuplicate,run_accessions,pc_pass_missing,pc_genome_covered_at_1x,Year_source,sort_AlfrescoStudyCode,IndividualGroup,AllSamplesThisIndividual,CountryCode,Lat,Long,Site_source
PM0001-C,215C,215C,215C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR012350,ERR012360,ERR012305",0.15139786050931692,0.9696,study,1001-PF-ML-DJIMDE,1,PM0001-C,ML,12.5546,-7.99805,https://www.malariagen.net/apps/pf3k/release_3/index.html#list_locations
PM0002-C,217C,217C,217C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR012520,ERR012331,ERR012327",0.28678549872221293,0.864,study,1001-PF-ML-DJIMDE,2,PM0002-C,ML,12.5546,-7.99805,https://www.malariagen.net/apps/pf3k/release_3/index.html#list_locations
PM0003-C,520C,520C,520C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR012492,ERR012457,ERR012532",0.1662372060768727,0.9586,study,1001-PF-ML-DJIMDE,3,PM0003-C,ML,12.5546,-7.99805,https://www.malariagen.net/apps/pf3k/release_3/index.html#list_locations
PM0004-C,A336TPI7C,A336TPI7C,A336TPI7C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR015438,ERR015329,ERR015422,ERR015315",0.11811536041300132,0.9648,study,1001-PF-ML-DJIMDE,4,PM0004-C,ML,12.5546,-7.99805,https://www.malariagen.net/apps/pf3k/release_3/index.html#list_locations
PM0005-C,B125,B125,B125,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,False,"ERR012392,ERR012318,ERR012271",0.22925169194416264,0.9155,study,1001-PF-ML-DJIMDE,35,PM0005-C,ML,12.5546,-7.99805,https://www.malariagen.net/apps/pf3k/release_3/index.html#list_locations


In [47]:
# File to send to Olivo
tbl_panoptes_samples_final.totsv(panoptes_final_metadata_fn, lineterminator='\n')

# Create spreadsheet for Vikki

In [20]:
tbl_study_summary = (
    (
        tbl_panoptes_samples_final
        .selecteq('InV5', 'True')
        .valuecounts(
            'AlfrescoStudyCode',
        )
        .rename('count', 'Total in 5.0')
        .cutout('frequency')
    ).outerjoin(
    (
        tbl_panoptes_samples_final
        .valuecounts(
            'AlfrescoStudyCode',
        )
        .rename('count', 'Total in 6.0')
        .cutout('frequency')
    ), key='AlfrescoStudyCode')
    .addfield('New in 6.0', lambda rec: rec['Total in 6.0'] if rec['Total in 5.0'] is None else rec['Total in 6.0'] - rec['Total in 5.0'])
    .outerjoin(
    (
        tbl_panoptes_samples_final
            .selectin('Year', ['-', 'N/A'])
        .valuecounts(
            'AlfrescoStudyCode',
        )
        .rename('count', 'Missing year')
        .cutout('frequency')
    ), key='AlfrescoStudyCode')
    .outerjoin(
    (
        tbl_panoptes_samples_final
            .selectin('Site', ['-'])
        .valuecounts(
            'AlfrescoStudyCode',
        )
        .rename('count', 'Missing site')
        .cutout('frequency')
    ), key='AlfrescoStudyCode')
    .replaceall(None, 0)
    .sort(['AlfrescoStudyCode'])
)
tbl_study_summary.toxlsx(study_summary_fn)
tbl_study_summary.displayall()


AlfrescoStudyCode,Total in 5.0,Total in 6.0,New in 6.0,Missing year,Missing site
1001-PF-ML-DJIMDE,96,96,0,0,0
1004-PF-BF-OUEDRAOGO,56,57,1,0,0
1006-PF-GM-CONWAY,79,79,0,0,0
1007-PF-TZ-DUFFY,50,50,0,0,0
1008-PF-SEA-RINGWALD,234,234,0,0,0
1009-PF-KH-PLOWE,5,5,0,0,0
1010-PF-TH-ANDERSON,107,108,1,0,0
1011-PF-KH-SU,40,41,1,0,0
1012-PF-KH-WHITE,2,2,0,0,0
1013-PF-PEGB-BRANCH,16,16,0,0,0


In [21]:
partner_columns = ['Sample', 'OxfordSrcCode', 'Site', 'Year', 'AllSamplesThisIndividual']

In [22]:
from pandas import ExcelWriter

studies = tbl_study_summary.values('AlfrescoStudyCode').list()
writer = ExcelWriter(samples_by_study_fn)
for study in studies:
    print(study)
    if tbl_study_summary.selecteq('AlfrescoStudyCode', study).values('Total in 5.0')[0] > 0:
        df = (
            tbl_panoptes_samples_final
            .selecteq('AlfrescoStudyCode', study)
            .selecteq('InV5', 'True')
            .cut(partner_columns)
            .todataframe()
        )
        sheet_name = "%s_old" % study[0:4]
        df.to_excel(writer, sheet_name, index=False)
    if tbl_study_summary.selecteq('AlfrescoStudyCode', study).values('New in 6.0')[0] > 0:
        df = (
            tbl_panoptes_samples_final
            .selecteq('AlfrescoStudyCode', study)
            .selecteq('InV5', 'False')
            .cut(partner_columns)
            .todataframe()
        )
        sheet_name = "%s_new" % study[0:4]
        df.to_excel(writer, sheet_name, index=False)
writer.save()
    

1001-PF-ML-DJIMDE
1004-PF-BF-OUEDRAOGO
1006-PF-GM-CONWAY
1007-PF-TZ-DUFFY
1008-PF-SEA-RINGWALD
1009-PF-KH-PLOWE
1010-PF-TH-ANDERSON
1011-PF-KH-SU
1012-PF-KH-WHITE
1013-PF-PEGB-BRANCH
1014-PF-SSA-SUTHERLAND
1015-PF-KE-NZILA
1016-PF-TH-NOSTEN
1017-PF-GH-AMENGA-ETEGO
1018-PF-GB-NEWBOLD
1020-PF-VN-BONI
1021-PF-PG-MUELLER
1022-PF-MW-OCHOLLA
1023-PF-CO-ECHEVERRI-GARCIA
1024-PF-UG-BOUSEMA
1025-PF-KH-PLOWE
1026-PF-GN-CONWAY
1027-PF-KE-BULL
1031-PF-SEA-PLOWE
1044-PF-KH-FAIRHURST
1052-PF-TRAC-WHITE
1062-PF-PG-BARRY
1083-PF-GH-CONWAY
1093-PF-CM-APINJOH
1094-PF-GH-AMENGA-ETEGO
1095-PF-TZ-ISHENGOMA
1096-PF-GH-GHANSAH
1097-PF-ML-MAIGA
1098-PF-ET-GOLASSA
1100-PF-CI-YAVO
1101-PF-CD-ONYAMBOKO
1102-PF-MG-RANDRIANARIVELOJOSIA
1103-PF-PDN-GMSN-NGWA
1107-PF-KEN-KAMAU
1125-PF-TH-NOSTEN
1127-PF-ML-SOULEYMANE
1131-PF-BJ-BERTIN
1133-PF-LAB-MERRICK
1134-PF-ML-CONWAY
1135-PF-SN-CONWAY
1136-PF-GM-NGWA
1137-PF-GM-DALESSANDRO
1138-PF-CD-FANELLO
1141-PF-GM-CLAESSENS
1145-PF-PE-GAMBOA
1147-PF-MR-CONWAY
1151-PF-GH-AME

# Create new Panaoptes file

In [23]:
tbl_sample_6_0 = (
    etl
    .fromtsv(sample_6_0_fn)
    .cutout('study')
    .cutout('source_code')
    .cutout('run_accessions')
)
print(len(tbl_sample_6_0.data()))
print(len(tbl_sample_6_0.distinct('sample').data()))
tbl_sample_6_0

7182
7182


sample,center_name,pc_bases_callable,pc_bases_no_coverage,pc_bases_low_coverage,pc_bases_excessive_coverage,pc_bases_poor_mapping_quality,pc_bases_ref_n,bases_of_1X_coverage,bases_of_2X_coverage,bases_of_5X_coverage,pc_genome_covered_at_1x,pc_genome_covered_at_5x,mean_coverage,mean_insert_size,sd_insert_size,avg_read_length,reads,reads_mapped,reads_mapped_and_paired,reads_properly_paired,reads_qc_failed,pairs_on_different_chromosomes,non_primary_alignments,pc_reads_mapped,pc_mapped_reads_properly_paired,pc_pairs_on_different_chromosomes,pc_non_primary_alignments,pc_reads_qc_failed,num_variants,num_pass_variants,num_missing,num_pass_missing,num_called,num_pass_called,num_hom_ref,num_het,num_pass_het,num_hom_alt,num_pass_hom_alt,num_pass_non_ref,num_biallelic_het,num_biallelic_hom_alt,num_spanning_del_het,num_spanning_del_hom_alt,num_multiallelic_het,num_multiallelic_hom_alt,num_snp_het,num_snp_hom_alt,num_snp,num_indel_het,num_indel_hom_alt,num_indel,num_ins_het,num_ins_hom_alt,num_ins,num_del_het,num_del_hom_alt,num_del,num_coding_het,num_coding_hom_alt,num_coding,num_hq_snp_called,num_hq_snp_hom_ref,num_hq_snp_het,num_hq_snp_hom_alt,num_vhq_snp_called,num_vhq_snp_hom_ref,num_vhq_snp_het,num_vhq_snp_hom_alt,num_singleton,num_biallelic_singleton,num_hq_snp_singleton,num_vhq_snp_singleton,num_bi_nonsynonymous,num_bi_synonymous,num_frameshift,num_inframe,num_bi_frameshift,num_bi_inframe,num_hq_frameshift,num_hq_inframe,num_bi_frameshift_snpeff,num_bi_inframe_snpeff,num_bi_transition,num_bi_transversion,num_bi_AT_to_AT,num_bi_CG_to_CG,num_bi_AT_to_CG,num_bi_CG_to_AT,num_phased,num_phased_non_ref,num_phased_hom_ref,num_phased_missing,num_GQ_30,num_het_GQ_30,num_hom_alt_GQ_30,num_GQ_99,num_het_GQ_99,num_hom_alt_GQ_99,pc_pass,pc_missing,pc_pass_missing,pc_het,pc_pass_het,pc_hq_snp_het,pc_vhq_snp_het,pc_hom_alt,pc_pass_hom_alt,pc_snp,pc_biallelic,pc_spanning_del,pc_mutliallelic,pc_ins,pc_coding,pc_bi_nonsynonymous,pc_frameshift,pc_bi_frameshift,pc_hq_frameshift,pc_bi_frameshift_snpeff,pc_bi_transition,pc_bi_AT_to_AT,pc_bi_CG_to_CG,pc_bi_AT_to_CG,pc_bi_CG_to_AT,pc_phased,pc_phased_hom_ref,pc_phased_missing,pc_GQ_30,pc_het_GQ_30,pc_hom_alt_GQ_30,pc_GQ_99,pc_het_GQ_99,pc_hom_alt_GQ_99,mean_GQ,mean_GQ_hom_ref,mean_GQ_het,mean_GQ_hom_alt,mean_DP,mean_DP_hom_ref,mean_DP_het,mean_DP_hom_alt,mean_indel_len,total_indel_len
FP0008-C,SC,0.8216,0.0265,0.0942,0.0,0.0577,0.0,22897930,22494301,21155419,0.9814,0.9067,34.09,232.6,81.3,99,8893235,7876872,7836096,7377894,0,208885,107847,0.8857,0.9367,0.0536,0.0137,0.0,6051696,3114760,705552,214960,5346144,2899800,5173409,75703,30237,97032,65157,95394,3556,5476,10087,30724,16594,28957,9188,19951,29139,21049,21049,42098,4105,4728,8833,13145,31726,44871,13744,17108,30852,1042923,1037626,2260,3037,83095,79328,1295,2472,396,251,142,11,3510,1754,6589,7780,39,62,0,5,32,62,4656,3837,1752,472,3172,3097,20733,17693,2953,87,2098968,21382,28661,1195439,11750,8487,0.5424096320637828,0.11658748225290894,0.06901334292208709,0.01416029946069541,0.01042727084626526,0.0021669864409932468,0.015584571875564113,0.01814990393075832,0.022469480653838195,0.3054594628592993,0.09468100719122796,0.4278151665723211,0.47750382623645093,0.16447564427230746,0.3234165670796905,0.6667933130699089,0.4585566149349294,0.38613861386138615,0.0,0.3404255319148936,0.548216178028965,0.20628753090780644,0.055575179559637346,0.3734840456846815,0.36465324384787473,0.18547288089397657,0.14242994260357883,0.004196208942265953,0.723831988412994,0.7071468730363462,0.43987599183510595,0.41224877577763985,0.3885967523233125,0.1302546157742069,58.409232172,63.3459959079,60.7741508748,37.5281397241,23.9188714379,25.8241581283,23.5613652148,15.0279325322,-0.188319164237,-586569
FP0009-C,SC,0.8885,0.0127,0.027,0.0,0.0718,0.0,23176020,23074498,22753562,0.9933,0.9752,115.68,249.4,85.3,99,28121601,27340516,27190475,25462624,0,781402,401309,0.9722,0.9313,0.0578,0.0147,0.0,6051696,3114760,459029,84770,5592667,3029990,5372312,80727,19396,139628,101341,120737,149

In [41]:
tbl_panoptes_samples = (
    tbl_panoptes_samples_final
    .addfield('PreferredSample', lambda rec: rec['DiscardAsDuplicate'] == 'False')
    .convert('Year', lambda v: '', where=lambda r: r['PreferredSample'] == False)
    .convert('Date', lambda v: '', where=lambda r: r['PreferredSample'] == False)
    .cutout('DiscardAsDuplicate')
    .join(tbl_sample_6_0, lkey='Sample', rkey='sample')
    .sort(['AlfrescoStudyCode', 'Sample'])
)
print(len(tbl_panoptes_samples.data()))
print(len(tbl_panoptes_samples.distinct('Sample').data()))
tbl_panoptes_samples

7182
7182


Sample,OxfordSrcCode,OxfordDonorCode,Individual_ID,AlfrescoStudyCode,Type,InV5.1,InV5,SubContinentCode,SubContinentName,RegionCode,CountryCode,CountryName,Site,Year,Date,HasDuplicate,run_accessions,pc_pass_missing,pc_genome_covered_at_1x,Year_source,sort_AlfrescoStudyCode,IndividualGroup,AllSamplesThisIndividual,CountryCode,Lat,Long,Site_source,PreferredSample,center_name,pc_bases_callable,pc_bases_no_coverage,pc_bases_low_coverage,pc_bases_excessive_coverage,pc_bases_poor_mapping_quality,pc_bases_ref_n,bases_of_1X_coverage,bases_of_2X_coverage,bases_of_5X_coverage,pc_genome_covered_at_1x,pc_genome_covered_at_5x,mean_coverage,mean_insert_size,sd_insert_size,avg_read_length,reads,reads_mapped,reads_mapped_and_paired,reads_properly_paired,reads_qc_failed,pairs_on_different_chromosomes,non_primary_alignments,pc_reads_mapped,pc_mapped_reads_properly_paired,pc_pairs_on_different_chromosomes,pc_non_primary_alignments,pc_reads_qc_failed,num_variants,num_pass_variants,num_missing,num_pass_missing,num_called,num_pass_called,num_hom_ref,num_het,num_pass_het,num_hom_alt,num_pass_hom_alt,num_pass_non_ref,num_biallelic_het,num_biallelic_hom_alt,num_spanning_del_het,num_spanning_del_hom_alt,num_multiallelic_het,num_multiallelic_hom_alt,num_snp_het,num_snp_hom_alt,num_snp,num_indel_het,num_indel_hom_alt,num_indel,num_ins_het,num_ins_hom_alt,num_ins,num_del_het,num_del_hom_alt,num_del,num_coding_het,num_coding_hom_alt,num_coding,num_hq_snp_called,num_hq_snp_hom_ref,num_hq_snp_het,num_hq_snp_hom_alt,num_vhq_snp_called,num_vhq_snp_hom_ref,num_vhq_snp_het,num_vhq_snp_hom_alt,num_singleton,num_biallelic_singleton,num_hq_snp_singleton,num_vhq_snp_singleton,num_bi_nonsynonymous,num_bi_synonymous,num_frameshift,num_inframe,num_bi_frameshift,num_bi_inframe,num_hq_frameshift,num_hq_inframe,num_bi_frameshift_snpeff,num_bi_inframe_snpeff,num_bi_transition,num_bi_transversion,num_bi_AT_to_AT,num_bi_CG_to_CG,num_bi_AT_to_CG,num_bi_CG_to_AT,num_phased,num_phased_non_ref,num_phased_hom_ref,num_phased_missing,num_GQ_30,num_het_GQ_30,num_hom_alt_GQ_30,num_GQ_99,num_het_GQ_99,num_hom_alt_GQ_99,pc_pass,pc_missing,pc_pass_missing,pc_het,pc_pass_het,pc_hq_snp_het,pc_vhq_snp_het,pc_hom_alt,pc_pass_hom_alt,pc_snp,pc_biallelic,pc_spanning_del,pc_mutliallelic,pc_ins,pc_coding,pc_bi_nonsynonymous,pc_frameshift,pc_bi_frameshift,pc_hq_frameshift,pc_bi_frameshift_snpeff,pc_bi_transition,pc_bi_AT_to_AT,pc_bi_CG_to_CG,pc_bi_AT_to_CG,pc_bi_CG_to_AT,pc_phased,pc_phased_hom_ref,pc_phased_missing,pc_GQ_30,pc_het_GQ_30,pc_hom_alt_GQ_30,pc_GQ_99,pc_het_GQ_99,pc_hom_alt_GQ_99,mean_GQ,mean_GQ_hom_ref,mean_GQ_het,mean_GQ_hom_alt,mean_DP,mean_DP_hom_ref,mean_DP_het,mean_DP_hom_alt,mean_indel_len,total_indel_len
PM0001-C,215C,215C,215C,1001-PF-ML-DJIMDE,DNA,True,True,WAF,West Africa,ML,ML,Mali,Kolle,2007,2007,False,"ERR012350,ERR012360,ERR012305",0.15139786050931692,0.9696,study,1001-PF-ML-DJIMDE,1,PM0001-C,ML,12.5546,-7.99805,https://www.malariagen.net/apps/pf3k/release_3/index.html#list_locations,True,SC,0.7954,0.0326,0.0971,0.0,0.075,0.0,22622963,22095176,20666609,0.9696,0.8857,68.9,170.1,75.8,36,48453780,43775226,42698980,40039536,0,427250,0,0.9034,0.9147,0.0209,0.0,0.0,6051696,3114760,901476,471568,5150220,2643192,5101218,33545,9209,15457,7984,17193,4141,3124,1637,1879,3431,2981,6101,4665,10766,3108,3108,6216,469,382,851,1719,2046,3765,5580,3942,9522,1042114,1037173,2850,2091,82823,79312,1649,1862,302,197,159,2,3290,1627,713,1005,19,52,0,4,17,52,3884,3121,1438,412,2620,2535,2810,2647,160,3,1946160,8273,5233,1321470,6801,3176,0.5132192411197968,0.14896253876599222,0.15139786050931692,0.006513313994353639,0.0034840450485624955,0.002734825556513011,0.019909928401530975,0.0030012310153741005,0.0030205902560237773,0.6261850753213517,0.42255569126970277,0.2045018321409876,0.3729424765893096,0.18435875216637782,0.5538300471121969,0.6691071791742933,0.4150174621653085,0.2676056338028169,0.0,0.2463768115942029,0.5544610992148465,0.2052819414703783,0.05881513204853676,0.3740185581727338,0

In [42]:
tbl_panoptes_samples.valuecounts('DiscardAsDuplicate')

FieldSelectionError: selection is not a field or valid field index: 'DiscardAsDuplicate'

FieldSelectionError: selection is not a field or valid field index: 'DiscardAsDuplicate'

In [43]:
tbl_panoptes_samples.valuecounts('PreferredSample')

PreferredSample,count,frequency
True,6696,0.9323308270676691
False,486,0.06766917293233082


In [44]:
tbl_panoptes_samples.valuecounts('Year').displayall()

Year,count,frequency
2013,1749,0.24352548036758562
2011,1242,0.17293233082706766
2014,812,0.11306042884990253
2012,742,0.10331384015594541
2008,638,0.08883319409635199
2010,558,0.07769423558897243
,486,0.06766917293233082
2007,235,0.032720690615427456
2009,219,0.030492898913951544
2015,109,0.01517683096630465


In [45]:
tbl_panoptes_samples.totsv(panoptes_samples_fn, lineterminator='\n')

After running the above, ran the following from oscar.well.ox.ac.uk

rsync -av rp7@malsrv2:/nfs/team112_internal/production/release_build/Pf/6_0_release_packages/Pf_60_samples_panoptes_20170206.txt /kwiat/2/grassi/sanger_mirror/nfs/team112_internal/production/release_build/Pf/6_0_release_packages/ >> ~/bin/log/Pf_60_Panoptes_20170206.log
